In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

# Problem:

Given customer data, explore:

1. customer behavior/preferences
1. analyze marketing campaigns - customer demographics/referral sources
1. evaluate effectiveness of referral strategies
1. determine which demographic groups are more likely to subscribe to certain subscriptions
1. identify trends in subscription types
1. user characteristics/behavior determine churn

-- Subscription-based, SaaS business questions are limited by given dataset.

# Data Dictionary:

* cust_id: A unique identifier for each customer.
* transaction_type: The type of transaction.
* transaction_date: The date when the transaction was made.
* subscription_type: The type of subscription, which can be BASIC, MAX, or PRO.
* subscription_price: The price of the subscription.
* customer_gender: The gender of the customer, which can be Male or Female.
* age_group: The age group of the customer.
* customer_country: The country where the customer is located.
* referral_type: The type of referral, which can be Google Ads, Facebook, Display, or others.

# Data

In [ ]:
df = pd.read_csv('/kaggle/input/performance-of-a-subscription-business/Customer_Subscription_And_Transaction_Details.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

This is a clean dataset with multiple lines per customer because there are multiple transactions per customer recorded.

In [ ]:
# Span of data: 2 full years
df.transaction_date.min(), df.transaction_date.max()

In [ ]:
# Let's take a look at some initial customer demographic data by filtering to first initial tx/customer id
cust_init = df.loc[df['transaction_type'] == 'initial']
cust_init

In [ ]:
#checking to see how many unique customers there are matches table output
df['cust_id'].nunique()

In [ ]:
#When did they become subscribers?
tx_date_ct = df.groupby(['transaction_date']).size().reset_index(name='counts')
tx_date_ct.plot.bar(x = 'transaction_date', y='counts')

In [ ]:
#Let's see if there is any month that overpowerd any other regarding intiial sub's
cust_init['tx_initial_month'] = pd.DatetimeIndex(cust_init['transaction_date']).month
tx_date_ct_month = cust_init.groupby(['tx_initial_month']).size().reset_index(name='counts')
tx_date_ct_month.plot.bar(x = 'tx_initial_month', y='counts')

In [ ]:
#What was most popular channel that caused subscriptions?
init_refs = cust_init.groupby(['referral_type']).size().reset_index(name='counts')
plt.pie(x=init_refs["counts"], labels=init_refs.referral_type, autopct='%1.0f%%')

In [ ]:
# Country that initial sign-up's came from
init_loc = cust_init.groupby(['customer_country']).size().reset_index(name='counts')
plt.pie(x=init_loc["counts"], labels=init_loc.customer_country, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
# Age Group that initial sign-up's came from
init_age = cust_init.groupby(['age_group']).size().reset_index(name='counts')
plt.pie(x=init_age["counts"], labels=init_age.age_group, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
# Gender that initial sign-up's came from
init_gender = cust_init.groupby(['customer_gender']).size().reset_index(name='counts')
plt.pie(x=init_gender["counts"], labels=init_gender.customer_gender, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
# Subscription type during initial sign-up
init_sub = cust_init.groupby(['subscription_type']).size().reset_index(name='counts')
plt.pie(x=init_sub["counts"], labels=init_sub.subscription_type, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
# Subscription type during initial sign-up
init_sub = cust_init.groupby(['subscription_type']).size().reset_index(name='counts')
plt.pie(x=init_sub["counts"], labels=init_sub.subscription_type, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
# Subscription price during initial sign-up
init_price = cust_init.groupby(['subscription_price']).size().reset_index(name='counts')
plt.pie(x=init_price["counts"], labels=init_price.subscription_price, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
grouped = cust_init.groupby(['subscription_type', 'subscription_price']).size().reset_index(name='counts')
grouped.sort_values('counts', ascending=False)

### Observations from Initial Sign-up's:
1. There was growth from beginning of 2020 to end of 2022. Growth is not continual increase though. Some dips in between, but not seasonal.
1. Not much difference month-to-month regardless of subscription year. Though June had the highest number of subscribers.
1. Most customers are coming from paid ads placed on Google and Facebook platforms.
1. Majority are coming from Sweden.
1. Biggest age demographic is 18-24 but all other age groups are not that far behind. So this subscription has a great spread across age groups.
1. Most customers are female but not highly dominant.
1. Basic and Pro subscriptions (lowest and middle tier) were most popular sub types.
1. Price range for sub's were mostly between 33 and 85, which is a huge range and does not map to 3 different sub tiers. There are 3 different prices per subscription type. And the most dominant initial subscription were from the lowest tiers from Basic Subscription Type with lowest Pro tier following in 3rd place.

In [ ]:
# Now let's look at different populations compared against each other - those who upgrade (initial > upgrade) VERSUS those who downgrade (inital>reduction / initial/churn / reduction>churn / upgrade/reduction / upgrade>churn)
counts = df['cust_id'].value_counts()
df[df['cust_id'].isin(counts.index[counts > 1])].head(20)

Just did data audit and realized that some transaction data may be duplicated. Need to only keep rows that are upgrades/downgrades and exact number of churns as there are sign-up's from initial to upgrade/downgrade

DELETE LAST ROW FROM USERS WHO HAVE 5 TRANSACTIONS. Tx data in general looks odd.

In [ ]:
# Importing New Dataset (Max transactions/customer is 4)
df2 = pd.read_csv('/kaggle/input/sass-cust-data/saas_cust_tx_edit.csv')
df2

### Subsequent Transactions' Demographics

We want to see if there is some common denominator with upgrades/churn by looking at some qualitative customer data

In [ ]:
sub_tx_upgrade = pd.read_csv('/kaggle/input/sass-cust-data/subsequent_upgrade.csv')
sub_tx_churn = pd.read_csv('/kaggle/input/sass-cust-data/subsequent_churn.csv')

## Upgrades

In [ ]:
up_gender_cnt = sub_tx_upgrade.groupby(['customer_gender1']).size().reset_index(name='counts')
plt.pie(x=up_gender_cnt["counts"], labels=up_gender_cnt.customer_gender1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
up_age_cnt = sub_tx_upgrade.groupby(['age_group1']).size().reset_index(name='counts')
plt.pie(x=up_age_cnt["counts"], labels=up_age_cnt.age_group1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
up_country_cnt = sub_tx_upgrade.groupby(['customer_country1']).size().reset_index(name='counts')
plt.pie(x=up_country_cnt["counts"], labels=up_country_cnt.customer_country1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
up_ref_cnt = sub_tx_upgrade.groupby(['referral_type1']).size().reset_index(name='counts')
plt.pie(x=up_ref_cnt["counts"], labels=up_ref_cnt.referral_type1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

## Churn

In [ ]:
churn_gender_cnt = sub_tx_churn.groupby(['customer_gender1']).size().reset_index(name='counts')
plt.pie(x=churn_gender_cnt["counts"], labels=churn_gender_cnt.customer_gender1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
churn_age_cnt = sub_tx_churn.groupby(['age_group1']).size().reset_index(name='counts')
plt.pie(x=churn_age_cnt["counts"], labels=churn_age_cnt.age_group1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
churn_country_cnt = sub_tx_churn.groupby(['customer_country1']).size().reset_index(name='counts')
plt.pie(x=churn_country_cnt["counts"], labels=churn_country_cnt.customer_country1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

In [ ]:
churn_ref_cnt = sub_tx_churn.groupby(['referral_type1']).size().reset_index(name='counts')
plt.pie(x=churn_ref_cnt["counts"], labels=churn_ref_cnt.referral_type1, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)

# Summary:

* Not that many differences between initial sign-up demographics vs upgrade/churn demographics except for the following:

1. upgrades were a little more bias towards 18-24
2. churn was heavy amongst those coming from Finland and Denmark